# Rounding concerns

We have a small concern with numerical stability (really numerical conditioning).  In the computations leading up to the LCM model, we compute each galaxy's $\phi$ and $\beta$, which are quite small.  We then do some other computations with these as inputs.  Starting from these, we in particular compute

$$\psi_{flat} = eTsiFlat(\beta) = \sqrt{\frac{1 + \beta}{1 - \beta}}$$

$$\psi_{curve} = eTsiCurve(\phi_{MW}, \phi_{gal}) = \sqrt{\frac{1-2\phi_{MW}}{1-2\phi_{gal}}}$$

These are both approximately $1$, with a tiny deviation.  We then take these and compute

$$v_2 = \frac{\psi_{flat} + \psi_{curve}}{\psi_{flat} - \psi_{curve}}$$

$$v_1 = 1 - \frac{2}{\psi_{curve} + \frac{1}{\psi_{curve}}}$$

As far as I can tell, this is the only thing they're ever used for.

Since the $\psi$'s are close to $1$, subtracting them is a recipe for floating point (finite decimal place) issues.  We can work around this by being slightly more clever.

First, we should be using `np.float64` variables.  I suspect we're already doing this, but it's worth double checking.

We don't actually want to carry around $\psi$, we want $\psi - 1$, since the deviation from $1$ is what we care about (and is much more computationally accurate).  Note that the following techniques will work fine for $\beta$ and $\phi$ relatively small (say from $-0.2$ to $0.2$ or so), but will have issues as they get close to $1$.  But note this isn't a Taylor Series or some sort of approximation, it's exact.

We can change things around by doing some mathematical manipulation (this trick of completing the square is often useful in these sorts of situations):

$$\Delta\psi_{flat} = eTsiFlat(\beta) - 1 = \sqrt{\frac{1 + \beta}{1 - \beta}} - 1 = 
\left(\sqrt{\frac{1 + \beta}{1 - \beta}} - 1\right)\cdot
\frac{\sqrt{\frac{1 + \beta}{1 - \beta}} + 1}{\sqrt{\frac{1 + \beta}{1 - \beta}} + 1} = 
\frac{\frac{2\beta}{1 - \beta}}{\sqrt{\frac{1 + \beta}{1 - \beta}} + 1}$$

Pulling the same trick here (I'm not writing out the algebra):

$$\Delta\psi_{curve} = eTsiCurve(\phi_{MW}, \phi_{gal}) - 1 = \sqrt{\frac{1-2\phi_{MW}}{1-2\phi_{gal}}} - 1 = 
\frac{\frac{2\phi_{gal} - 2\phi_{MW}}{1 - 2\phi_{gal}}}{\sqrt{\frac{1-2\phi_{MW}}{1-2\phi_{gal}}} + 1}$$

This latter one still has issues if $\phi_{MW} \approx \phi_{gal}$, but I'm pretty sure that's an intrinsic feature of the LCM.

In [11]:
import numpy as np

#Current functions:

def eTsiFlat(beta):
  return np.sqrt((1 + beta)/(1 - beta))

def eTsiCurve(MW_phi, Other_phi):
  return np.sqrt((1 - 2*MW_phi)/(1 - 2*Other_phi))

#New definitions, old ones minus 1

def deltaETsiFlat(beta):
    return 2*beta / (1 - beta) / (np.sqrt((1+beta) / (1-beta)) + 1)

def deltaETsiCurve(MW_phi, Other_phi):
    return (2*Other_phi - 2*MW_phi) / (1 - 2*Other_phi) / (np.sqrt((1 - 2*MW_phi)/(1 - 2*Other_phi)) + 1)

Let's check the flat over a broad range of $\beta$'s

In [30]:
b = np.logspace(-1,-20, 20)
print(eTsiFlat(b) - 1)
print()
print(deltaETsiFlat(b))

[1.05541597e-01 1.00505038e-02 1.00050050e-03 1.00005001e-04
 1.00000500e-05 1.00000050e-06 1.00000005e-07 9.99999994e-09
 1.00000008e-09 1.00000008e-10 1.00000008e-11 1.00008890e-12
 9.99200722e-14 9.99200722e-15 1.11022302e-15 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]

[1.05541597e-01 1.00505038e-02 1.00050050e-03 1.00005001e-04
 1.00000500e-05 1.00000050e-06 1.00000005e-07 1.00000001e-08
 1.00000000e-09 1.00000000e-10 1.00000000e-11 1.00000000e-12
 1.00000000e-13 1.00000000e-14 1.00000000e-15 1.00000000e-16
 1.00000000e-17 1.00000000e-18 1.00000000e-19 1.00000000e-20]


Ok, that looks quite good, the old way bottoms out and the new one is fine (and they agree completely for larger values of $\beta$ where we know they both should work).

Similarly for `eTsiCurve`, we pick a range of $\phi$'s.  We could be careful and pick lots of values to compare, but I'm goig to take that same range of $\beta$'s for the Milky Way $\phi$ and multiply by a random number between $0.3$ and $3$ at each point.

In [29]:
phi_MW = b
phi_other = b * np.random.uniform(0.3,3,size=20)
print(eTsiCurve(phi_MW, phi_other) - 1)
print()
print(deltaETsiCurve(phi_MW, phi_other))

[ 7.22283829e-02  1.84405404e-02 -3.36547394e-04  2.04082655e-05
  5.15638263e-06  5.82142514e-07 -2.22980290e-08  1.97839840e-08
  1.31871269e-09  8.80167050e-11  1.29836142e-11 -6.01185768e-13
  2.33146835e-14 -4.55191440e-15  1.33226763e-15  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]

[ 7.22283829e-02  1.84405404e-02 -3.36547394e-04  2.04082655e-05
  5.15638263e-06  5.82142514e-07 -2.22980290e-08  1.97839838e-08
  1.31871279e-09  8.80168466e-11  1.29836034e-11 -6.01235613e-13
  2.32378509e-14 -4.54176931e-15  1.40641682e-15  1.04610217e-16
  1.06885144e-17  1.51686955e-18  1.80802849e-19  1.66964874e-20]


Again they agree for larger numbers, then start diverging when the numerical precision starts biting us for smaller $\phi$ values.  Also note that the $\phi$ is quite tiny before this is a problem, since we're using 64-bit floats

Now for actually using them, we'll need to re-write the $v_1$ and $v_2$ calculations.

$$v_2 = \frac{\psi_{flat} + \psi_{curve}}{\psi_{flat} - \psi_{curve}} = 
\frac{2 + \Delta\psi_{flat} + \Delta\psi_{curve}}{\Delta\psi_{flat} - \Delta\psi_{curve}}$$

$$v_1 = 1 - \frac{2}{\psi_{curve} + \frac{1}{\psi_{curve}}} = 1 - \frac{2\psi_{curve}}{\psi_{curve}^2 + 1} = 
\frac{\psi_{curve}^2 + 1 - 2\psi_{curve}}{\psi_{curve}^2 + 1} = 
\frac{(\psi_{curve} - 1)^2}{\psi_{curve}^2 + 1} = \frac{\Delta\psi_{curve}^2}{(1 + \Delta\psi_{curve})^2 + 1}$$

In [32]:
#Current functions:

def v1(eTsiCurve):
  num = 2
  den = eTsiCurve + 1/eTsiCurve
  return 1 - (num/den)

def v2(eTsiFlat, eTsiCurve):
  num = eTsiFlat + eTsiCurve
  den = eTsiFlat - eTsiCurve
  return num/den

#New definitions, now taking deltaETsi

def v1_new(deltaETsiCurve):
  num = deltaETsiCurve**2
  den = (1 + deltaETsiCurve)**2 + 1
  return num / den

def v2_new(deltaETsiFlat, deltaETsiCurve):
  num = 2 + deltaETsiFlat + deltaETsiCurve
  den = deltaETsiFlat - deltaETsiCurve
  return num/den

And we'll just feed in to them the stuff we computed before, let's start with $v_1$

In [34]:
eTcurve = eTsiCurve(phi_MW, phi_other)
deltaETcurve = deltaETsiCurve(phi_MW, phi_other)

print(v1(eTcurve))
print()
print(v1_new(deltaETcurve))

[2.42685170e-03 1.66920284e-04 5.66511370e-08 2.08244533e-10
 1.32940325e-11 1.69420034e-13 2.22044605e-16 2.22044605e-16
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]

[2.42685170e-03 1.66920284e-04 5.66511369e-08 2.08244400e-10
 1.32940723e-11 1.69444855e-13 2.48601054e-16 1.95703005e-16
 8.69501705e-19 3.87348264e-21 8.42869789e-23 1.80742131e-25
 2.69998858e-28 1.03138342e-29 9.89004130e-31 5.47164877e-33
 5.71221701e-35 1.15044662e-36 1.63448350e-38 1.39386346e-40]


Again, the pattern we expect: nearly perfect agrement at large numbers, and increasing discrepancy as the old method falters.

And for $v_2$

In [35]:
eTflat = eTsiFlat(b)
deltaETflat = deltaETsiFlat(b)

print(v2(eTflat, eTcurve))
print()
print(v2_new(deltaETflat, deltaETcurve))

[ 6.53725571e+01 -2.41773801e+02  1.49632931e+03  2.51281716e+04
  4.12913398e+05  4.78631892e+06  1.63534933e+07 -2.04415708e+08
 -6.27524587e+09  1.66898889e+11 -6.70328143e+11  1.24900496e+12
  2.61078239e+13  1.37514492e+14 -9.00719925e+15             inf
             inf             inf             inf             inf]

[ 6.53725571e+01 -2.41773801e+02  1.49632931e+03  2.51281716e+04
  4.12913398e+05  4.78631892e+06  1.63534933e+07 -2.04415714e+08
 -6.27524246e+09  1.66900976e+11 -6.70330376e+11  1.24903542e+12
  2.60545076e+13  1.37534846e+14 -4.92105622e+15 -4.33819039e+17
 -2.90480486e+18 -3.86944830e+18 -2.47516026e+19 -2.98664042e+20]


/Users/rott/miniconda3/envs/data3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


And the same pattern emerges, with the extra interesting wrinkle that they diverge as the curves get small, but the new one handles it more gracefully (mostly because the old one was being fed zeros for these values at the end)